In [1]:
from os.path import join, splitext, basename
import glob
from datetime import datetime

import pandas as pd
import numpy as np

import sys
sys.path.append('/home/alexandrearaujo/libraries/faiss/')
import faiss

In [ ]:
list_images = glob.glob(join('/media/hdd1/kaggle/landmark-retrieval-challenge/index_rescale/*'))
filenames_index = set([splitext(basename(path))[0] for path in list_images])

In [ ]:
list_images = glob.glob(join('/media/hdd1/kaggle/landmark-retrieval-challenge/feature_index_rescale_sift_pkl/*'))

In [4]:
import pickle

def pickle_load(path):
  """function to load pickle object"""
  with open(path, 'rb') as f:
    return pickle.load(f)

def _pickle_dump(file, path):
  """function to dump picke object"""
  with open(path, 'wb') as f:
    pickle.dump(file, f, -1)

def _get_new_name(path):
  """rename file if file already exist"""
  i = 0
  new_path = path
  while isfile(new_path):
    ext = splitext(path)[1]
    new_path = path.replace(ext, '_{}{}'.format(i, ext))
    i += 1
  return new_path

def pickle_dump(file, path, force=False):
  """dump a file without deleting an existing one"""
  if force:
    _pickle_dump(file, path)
  elif not force:
    new_path = _get_new_name(path)
    _pickle_dump(file, new_path)

In [ ]:
count = 0
start = datetime.now()
for package in list_images:
    for filename, loc, des in pickle_load(package):
        count += 1
        if count % 10000 == 0 and count != 0:
            print('10000 images loaded in {}'.format((datetime.now() - start).total_seconds()))
            start = datetime.now()
        if filename not in filenames_index:
            print(filename)

In [ ]:
df = pd.read_csv('../submit/9_flatindex_sift_vlad/submit_2018-04-02_07.35.11_vlad_full_index_sift_k64.csv.gz')

In [ ]:
filenames = pickle_load('../submit/9_flatindex_sift_vlad/k64_v2/filenames_index.dump')

In [ ]:
len(filenames)

In [ ]:
index = faiss.read_index('../submit/9_flatindex_sift_vlad/k64_v2/index_full.dump')

In [ ]:
index.ntotal

In [ ]:
from collections import Counter
c = Counter(filenames)

In [ ]:
def map5metric(predicted, actual):
    predicted = np.array(predicted)
    actual = np.array(actual)
    metric = 0.
    for i in range(5):
        metric += np.sum(actual == predicted[:, i]) / (i + 1)
    metric /= actual.shape[0]
    return metric

In [ ]:
predicted = np.array([[5, 1, 2, 3, 4], [5, 1, 2, 3, 4]])
actual = np.array([[5], [1]]).flatten()

In [ ]:
map5metric(predicted, actual)

In [ ]:
a = np.random.random(size=(143, 10))
b = np.random.random(size=(34, 10))
c = np.random.random(size=(5, 10))

In [ ]:
a /= np.linalg.norm(a, ord=1, axis=1)[..., np.newaxis]

In [ ]:
np.linalg.norm(a, ord=1, axis=1)[..., np.newaxis]

In [ ]:
from itertools import product

In [ ]:
len(list(product(np.arange(64), np.arange(64), np.arange(64))))

In [ ]:
np.random.randint(0, 10, size=(20, 10)).max(axis=0)

In [ ]:
d = np.vstack((a, b, c))

In [ ]:
d -= d.mean()

In [56]:
import cv2
from scipy.stats import multivariate_normal
import pickle
from collections import defaultdict

In [72]:
def pickle_load(path):
  """function to load pickle object"""
  with open(path, 'rb') as f:
    return pickle.load(f, encoding='latin1')

def generate_gmm(descriptors, k):
  # Throw away gaussians with weights that are too small:
  em = cv2.ml.EM_create()
  em.setClustersNumber(k)
  em.trainEM(descriptors)

  means = np.float32(em.getMeans())
  covs = np.float32(em.getCovs())
  weights = np.float32(em.getWeights())[0]

  th = 1.0 / k
  means   = np.array([m for w, m in zip(weights, means) if w > th])
  covs    = np.array([c for w, c in zip(weights, covs) if w > th])
  weights = np.array([w for w in weights if w > th])
  return means, covs, weights

def likelihood_moment(x, ytk, moment):  
  x_moment = np.power(np.float32(x), moment) if moment > 0 else np.float32([1])
  return x_moment * ytk

def likelihood_statistics(samples, means, covs, weights):
  gaussians = {}
  s0, s1, s2 = defaultdict(float), defaultdict(float), defaultdict(float)
  samples = list(zip(range(0, len(samples)), samples))
  
  g = [multivariate_normal(mean=means[k], cov=covs[k]) for k in range(0, len(weights)) ]
  for index, x in samples:
    gaussians[index] = np.array([g_k.pdf(x) for g_k in g])

  for k in range(0, len(weights)):
    s0[k], s1[k], s2[k] = 0, 0, 0
    for index, x in samples:
      probabilities = np.multiply(gaussians[index], weights)
      probabilities = probabilities / np.sum(probabilities)
      s0[k] += likelihood_moment(x, probabilities[k], 0)
      s1[k] += likelihood_moment(x, probabilities[k], 1)
      s2[k] += likelihood_moment(x, probabilities[k], 2)

  return s0, s1, s2

def fisher_vector_weights(s0, s1, s2, means, covs, w, T):
  return np.array([((s0[k] - T * w[k]) / np.sqrt(w[k]) ) for k in range(0, len(w))])

def fisher_vector_means(s0, s1, s2, means, sigma, w, T):
  return np.array([(s1[k] - means[k] * s0[k]) / (np.sqrt(w[k] * sigma[k])) for k in range(0, len(w))])

def fisher_vector_sigma(s0, s1, s2, means, sigma, w, T):
  return np.array([(s2[k] - 2 * means[k]*s1[k]  + (means[k]*means[k] - sigma[k]) * s0[k]) / (np.sqrt(2*w[k])*sigma[k])  for k in range(0, len(w))])

def normalize(fisher_vector):
  v = np.sqrt(abs(fisher_vector)) * np.sign(fisher_vector)
  return v / np.sqrt(np.dot(v, v))

def fisher_vector(samples, means, covs, weights):
  s0, s1, s2 =  likelihood_statistics(samples, means, covs, weights)
  # print(s0, s1, s2)
  covs = np.float32([np.diagonal(covs[k]) for k in range(0, covs.shape[0])])
  a = fisher_vector_weights(s0, s1, s2, means, covs, weights, samples.shape[0])
  b = fisher_vector_means(s0, s1, s2, means, covs, weights, samples.shape[0])
  c = fisher_vector_sigma(s0, s1, s2, means, covs, weights, samples.shape[0])
  fv = np.concatenate([a.flatten(), b.flatten(), c.flatten()])
  fv = normalize(fv)
  return fv[np.newaxis]

In [6]:
index_dbe = pickle_load('../submit/18_dir_dbe/xindexes_dbe.pkl')

In [10]:
queries = pickle_load('../submit/18_dir_dbe/xqueries.pkl')

In [15]:
np.argsort(preds)[:100]

array([ 436194,  662229,  597939, 1039291,  182534,  825411,  463045,
       1049221,  218300, 1086121,  880338,  560583,  947842,  934271,
        754325,  647499,  597916,  859029,  736368,   34628,  855539,
        188085,  871438,  387865,  672381,    8721, 1046074,  840416,
        973477,  102968,  236747,  714451,  722399,  967302,  782387,
       1078800,  603032,  273053,  543397,  935744,  992608,   23543,
       1038023,  818211,  235192,  674329,  276928, 1051822,  489227,
        951975,  869879, 1038970,   40572,  275021,  360863,  581095,
        962267,  377420,  349132,  645584,  770733,  691403, 1006152,
        525904,  661144,  774333,   45603,  913737,  657765,  945503,
        526532,  656701,  848527, 1007600,   89246,  336650, 1021099,
        275593,   76985,  725735,   33446,  347490,  848594,  326557,
        525329,  202745,  374085,  886425,  813585, 1046228,  401471,
        821296,  427228,  881149,  199729,  518397,  815417,  774970,
        254959,  923

In [23]:
train_filenames = pickle_load('../submit/18_dir_dbe/filenames_index.dump')
test_filenames = pickle_load('../submit/18_dir_dbe/test_filenames.pkl')
predicted = pickle_load('../submit/18_dir_dbe/predicted.pkl')

def make_results(predicted):
    results = {}
    for query_ix, query_result in enumerate(predicted):
      results[test_filenames[query_ix]] = ' '.join(
        [train_filenames[ix] for ix in query_result if ix != -1])
    return results

results = make_results(predicted)

submit = pd.read_csv('../data/sample_submission.csv')
submit['images'] = submit['id'].apply(lambda ix: results.get(ix, ''))
submit_filename = 'submit_{}_dir_full_index_db20.csv.gz'.format(
            datetime.now().strftime('%Y-%m-%d_%H.%M.%S'))
submit.to_csv(submit_filename, index=False, compression='gzip')

In [31]:
size = 1
for x in (5, ):
    size *= x
print(size)

5
